# Basics 4: Create a dataset view on top of star schema

Star schema is one of the most practical ways to model a data warehouse. By splitting your datasets into separate fact and dimension tables you improve performance, avoid vast amount of duplicate data, and simplify ETL processes that keep all your data up-to-date.

However, some BI tools expect each dataset as a flat table. These BI tools typically do not have an internal data store at all, and therefore there is no disadvantages in using a flat model within the BI tool itself. You can make your database easier to use with such BI tools by providing your fundamental datasets as database views. This way a user of BI tool does not have to constitute the flat view in the BI tool himself by using SQL.

## Add a new database view

1. Execute `taito db add view_sales`.
2. Add the following content to the newly created files (**database/deploy/view_sales.sql**, **database/revert/view_sales.sql**, and **database/verify/view_sales.sql**).

```sql
-- Deploy view_sales to pg

BEGIN;

CREATE VIEW view_sales AS
SELECT
  -- product
  product.sku, product.name, product.category, product.subcategory,
  -- sale
  sale.quantity, sale.price,
  -- store
  store.name as store, store.postal_code, store.city, store.country,
  -- date
  date.date, date.day, date.day_of_week, date.day_of_week_name,
  date.month, date.month_name, date.quarter, date.year,
  -- unique key
  sale.key
FROM fact_sales sale,
     dim_dates date,
     dim_stores store,
     dim_products product
WHERE sale.date_key = date.key
AND sale.store_key = store.key
AND sale.product_key = product.key;

COMMIT;
```

```sql
-- Revert view_sales from pg

BEGIN;

DROP VIEW view_sales;

COMMIT;
```

```sql
-- Verify view_sales on pg

BEGIN;

SELECT key FROM load_sales LIMIT 1;

ROLLBACK;
```

3. Deploy the new database migration to local database with `taito db deploy`.

## Next lesson: [Basics 5 - Visualize data with Apache Superset](05.ipynb)